# Querying Model List
Getting the list of models on Kvant

https://documentation.kvant.cloud/products/maas/supported_models/#1-call-to-list-available-models:~:text=Different%20Sample%20Calls-,%23,-1.%20Call%20to


In [4]:
import os
import requests
import json
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

API_KEY = os.getenv("KVANT_API_KEY")
if not API_KEY:
    raise ValueError("KVANT_API_KEY not found in .env")

url = "https://maas.ai-2.kvant.cloud/v1/models"
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)

print("Status Code:", response.status_code)

try:
    data = response.json()  # parse JSON
    print(json.dumps(data, indent=2))  # pretty print
except ValueError:
    # If not JSON, just print text
    print(response.text)

Status Code: 200
{
  "data": [
    {
      "id": "inference-apertus-8b",
      "object": "model",
      "created": 1677610602,
      "owned_by": "openai"
    },
    {
      "id": "inference-apertus-70b",
      "object": "model",
      "created": 1677610602,
      "owned_by": "openai"
    },
    {
      "id": "inference-bge-m3",
      "object": "model",
      "created": 1677610602,
      "owned_by": "openai"
    },
    {
      "id": "inference-deepseekr1-70b",
      "object": "model",
      "created": 1677610602,
      "owned_by": "openai"
    },
    {
      "id": "inference-deepseekr1-670b",
      "object": "model",
      "created": 1677610602,
      "owned_by": "openai"
    },
    {
      "id": "inference-llama33-70b",
      "object": "model",
      "created": 1677610602,
      "owned_by": "openai"
    },
    {
      "id": "inference-llama4-maverick",
      "object": "model",
      "created": 1677610602,
      "owned_by": "openai"
    },
    {
      "id": "inference-llama4-scout-17b",

# Calling Chat Complete



In [ ]:
import os
import requests
import json
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

API_KEY = os.getenv("KVANT_API_KEY")
if not API_KEY:
    raise ValueError("KVANT_API_KEY not found in .env")

url = "https://maas.ai-2.kvant.cloud/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}",
}

payload = {
    "model": "inference-gemma-12b-it",
    "messages": [
        {"role": "user", "content": "How do I make sourdough bread?"}
    ],
    "temperature": 0.7,
}

response = requests.post(url, headers=headers, json=payload)

print("Status Code:", response.status_code)

try:
    data = response.json()
    print(json.dumps(data, indent=2))
except ValueError:
    print(response.text)


Status Code: 200
{
  "id": "chatcmpl-b568730b-02b6-486b-a373-afca05b73d6a",
  "created": 1759166236,
  "model": "hosted_vllm/infer-llama4-maverick-400b",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Making sourdough bread is a multi-step process that requires patience, persistence, and a little bit of luck. Here's a basic recipe and guide to get you started:\n\n**Step 1: Create Your Sourdough Starter**\n\nTo make sourdough bread, you'll need to create a sourdough starter, which is a naturally occurring mixture of wild yeast and bacteria. To create your starter, you'll need:\n\n* 1/2 cup of warm water (around 90\u00b0F to 100\u00b0F)\n* 1/2 cup of flour (use a high-protein flour like bread flour or all-purpose flour with a high protein content)\n\nMix the water and flour in a clean glass or ceramic container until they form a smooth, thick batter. Cover the contai